# Import and Settings

In [35]:
import numpy as np
import random
from scipy.linalg import eig
import time

## Settings and parameter

In [36]:
m = 0.15

# Define the tolerance for the eigenvalue 
# and eigenvector approximations 
# (i.e. the maximum allowed difference between 
# the approximations and the actual values) 
tol = 1e-10
  
# Define the maximum number of iterations 
max_iter = 1000

## Generate random directed graph

In [37]:
# generates a directed graph from a given number of nodes
def generate_random_graph(number_of_nodes):
    return [[random.randint(0, 1) if row != col else 0 for col in range(number_of_nodes)] for row in range(number_of_nodes)]

# Processing

## Dangling Nodes

In [38]:
def check_for_negatives(graph):
    for row in graph:
        for value in row:
            if value < 0:
                return True
    return False

In [39]:
def handling_dangly_nodes(graph):
    if check_for_negatives(graph):
        return
    test_graph = []
    for row in graph:
        if np.sum(row) == 0:
            test_graph.append([1/len(row) for _ in row])
        else:
            test_graph.append(row)
    return test_graph

## Converting Graph

In [40]:
# Assumes it is a square matrix
def convert_graph_to_link_matrix(graph):
    transformed_graph = []
    for row in graph:
        col = []
        sum_row = sum(row)
        for val in row:
            col.append(val / sum_row)
        transformed_graph.append(col)
    transformed_graph = np.array(transformed_graph)
    transformed_graph = transformed_graph.T
    return transformed_graph

## Dangling Links

In [41]:
def handling_dangly_links(graph):
    S = np.full((len(graph), len(graph)), 1/len(graph))
    return ((1 - m) * graph) + (m * S)

## Eigen Values and Vector

In [42]:
def find_eigen_vectors(matrix):
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = eig(matrix)

    # Define a tolerance level for eigenvalue proximity (e.g., 0.01)
    tolerance = 0.01

    # Find eigenvectors corresponding to eigenvalues close to 1
    eigenvectors_close_to_1 = []
    for i in range(len(eigenvalues)):
        if np.isclose(eigenvalues[i], 1, atol=tolerance):
            eigenvectors_close_to_1.append(eigenvectors[:, i])

    # Convert the list of eigenvectors to a NumPy array
    eigenvectors_close_to_1 = np.array(eigenvectors_close_to_1)

    eigenvectors_close_to_1 = eigenvectors_close_to_1[0]

    # Normalize
    eigenvectors_close_to_1 = [rank / sum(eigenvectors_close_to_1) for rank in eigenvectors_close_to_1]
    return eigenvectors_close_to_1

## Power Method

In [43]:
# Code taken from https://www.geeksforgeeks.org/power-method-determine-largest-eigenvalue-and-eigenvector-in-python/

def power_method(A):
    x = np.array([[1 for i in range(len(A))]]).T
    # Define the variable lam_prev to store the 
    # previous approximation for the largest eigenvalue 
    lam_prev = 0

    # Iteratively improve the approximations 
    # for the largest eigenvalue and eigenvector 
    # using the power method 
    for i in range(max_iter): 
        # Compute the updated approximation for the eigenvector 
        x = A @ x / np.linalg.norm(A @ x) 

        # Compute the updated approximation for the largest eigenvalue 
        lam = (x.T @ A @ x) / (x.T @ x) 

        # Check if the approximations have converged 
        if np.abs(lam - lam_prev) < tol: 
            break

        # Store the current approximation for the largest eigenvalue 
        lam_prev = lam 

    # largest eigenvalue and eigenvector 
    return x

# Running Page Rank

In [44]:
def page_rank(graph):
    start = time.time()
    A = handling_dangly_nodes(graph)
    A = convert_graph_to_link_matrix(A)
    A = handling_dangly_links(A)
    vector = find_eigen_vectors(A)
    end = time.time()
    total_time = end-start
    return vector, total_time

In [45]:
def page_rank_power(graph):
    start = time.time()
    A = handling_dangly_nodes(graph)
    A = convert_graph_to_link_matrix(A)
    A = handling_dangly_links(A)
    vector = power_method(A)
    end = time.time()
    total_time = end-start
    return vector, total_time

In [46]:
directed_graph = [[0,1,1,1],[0,0,1,1],[1,0,0,0],[1,0,1,0]]


In [52]:
vector, total_time = page_rank(directed_graph)
print('Calculating Eigenvector of Eigenvalue 1')
print(total_time, ' seconds')

# Create a list of tuples where each tuple contains the index and the corresponding value
indexed_array = list(enumerate(vector, start=1))

# Sort the list of tuples by the value in descending order
indexed_array.sort(key=lambda x: x[1], reverse=True)

# Print the sorted list along with index + 1
for index, value in indexed_array:
    print(f"Node #{index}: {value}")

0.0007050037384033203  seconds
Node #1: (0.3681506770476028+0j)
Node #3: (0.2879616285976069+0j)
Node #4: (0.20207833585796964+0j)
Node #2: (0.14180935849682083+0j)


In [53]:
vector, total_time = page_rank_power(directed_graph)
print('Using Power Method')
print(total_time, ' seconds')

# Create a list of tuples where each tuple contains the index and the corresponding value
indexed_array = list(enumerate(vector, start=1))

# Sort the list of tuples by the value in descending order
indexed_array.sort(key=lambda x: x[1], reverse=True)

# Print the sorted list along with index + 1
for index, value in indexed_array:
    print(f"Node #{index}: {value}")

0.010232210159301758  seconds
Node #1: [0.69648307]
Node #3: [0.54477802]
Node #4: [0.38230037]
Node #2: [0.26828096]
